In [1]:
"""

LICENSE MIT
2020
Guillaume Rozier
Website : http://www.covidtracker.fr
Mail : guillaume.rozier@telecomnancy.net

README:
This file contains scripts that download data from data.gouv.fr and then process it to build many graphes.

The charts are exported to 'charts/images/france'.
Data is download to/imported from 'data/france'.
Requirements: please see the imports below (use pip3 to install them).

"""

"\n\nLICENSE MIT\n2020\nGuillaume Rozier\nWebsite : http://www.covidtracker.fr\nMail : guillaume.rozier@telecomnancy.net\n\nREADME:\nThis file contains scripts that download data from data.gouv.fr and then process it to build many graphes.\n\nThe charts are exported to 'charts/images/france'.\nData is download to/imported from 'data/france'.\nRequirements: please see the imports below (use pip3 to install them).\n\n"

In [2]:
import pandas as pd
import plotly.express as px
from datetime import timedelta
import france_data_management as data
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import plotly
import cv2
PATH = "../../"
import locale
locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')

'fr_FR.UTF-8'

In [3]:
COULEUR_NON_VACCINES = "#C65102"
COULEUR_COMPLETEMENT_VACCINES = "#00308F"
COULEUR_PARTIELLEMENT_VACCINES = "#4777d6"

In [4]:
df_drees = pd.read_csv("https://data.drees.solidarites-sante.gouv.fr/explore/dataset/covid-19-resultats-issus-des-appariements-entre-si-vic-si-dep-et-vac-si/download/?format=csv&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B", sep=";")
df_drees = df_drees.sort_values(by="date")
df_drees = df_drees[df_drees["vac_statut"]!="Ensemble"]

In [5]:
df_drees_age = pd.read_csv("https://data.drees.solidarites-sante.gouv.fr/explore/dataset/covid-19-resultats-par-age-issus-des-appariements-entre-si-vic-si-dep-et-vac-si/download/?format=csv&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B", sep=";")
df_drees_age_all = df_drees_age.groupby(["date", "vac_statut", "age"]).sum().reset_index()
df_drees_age = df_drees_age.sort_values(by="date")
df_drees_age = df_drees_age[df_drees_age["vac_statut"]!="Ensemble"]
df_drees_age_lastday = df_drees_age[df_drees_age["date"] == df_drees_age["date"].max()]

In [6]:
df_drees_non_vaccines = df_drees[df_drees["vac_statut"]=="Non-vaccinés"]
df_drees_non_vaccines["effectif J-7"] = df_drees_non_vaccines["effectif J-7"].rolling(window=7).mean()

df_drees_completement_vaccines = df_drees[df_drees["vac_statut"].isin(["Vaccination complète",])].groupby("date").sum().reset_index()
df_drees_completement_vaccines["effectif J-7"] = df_drees_completement_vaccines["effectif J-7"].rolling(window=7).mean()

df_drees_partiellement_vaccines = df_drees[df_drees["vac_statut"].isin(["Primo dose récente", "Primo dose efficace"])].groupby("date").sum().reset_index()
df_drees_partiellement_vaccines["effectif J-7"] = df_drees_partiellement_vaccines["effectif J-7"].rolling(window=7).mean()

#df_drees_ensemble = df_drees[df_drees["vac_statut"]=="Ensemble"]
df_drees_ensemble = df_drees.groupby("date").sum().reset_index()

/Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
fig = go.Figure()
ages=df_drees_age_lastday.age.sort_values().unique()
ages_str = ["0 à 19 ans", "20 à 39 ans", "40 à 59 ans", "60 à 79 ans", "plus 80 ans"]

y=[]
for age in ages:
    data=df_drees_age[(df_drees_age.vac_statut=="Non-vaccinés") \
                      & (df_drees_age.age==age)]
    y+=[data["HC"].rolling(window=7).mean().values[-1]/data["effectif J-7"].values[-1]*10000000]

fig.add_trace(go.Bar(
    x=ages_str,
    y=y,
    marker=dict(color=COULEUR_NON_VACCINES),
    text=[int(round(val)) for val in y],
    textposition='outside',
    name="Non vaccinés"
))

y=[]
for age in ages:
    data=df_drees_age[df_drees_age.vac_statut.isin(["Primo dose récente", "Primo dose efficace"])].groupby(["date", "age"]).sum().reset_index()
    data=data[(data.age==age)]
    y+=[data["HC"].rolling(window=7).mean().values[-1]/data["effectif J-7"].values[-1]*10000000]

fig.add_trace(go.Bar(
    x=ages_str,
    y=y,
    marker=dict(color=COULEUR_PARTIELLEMENT_VACCINES),
    text=[int(round(val)) for val in y],
    textposition='outside',
    name="Partiellement vaccinés")
             )

y=[]
for age in ages:
    data=df_drees_age[(df_drees_age.vac_statut=="Vaccination complète") &\
                             (df_drees_age.age==age)]
    y+=[data["HC"].rolling(window=7).mean().values[-1]/data["effectif J-7"].values[-1]*10000000]

fig.add_trace(go.Bar(
    x=ages_str,
    y=y,
    marker=dict(color=COULEUR_COMPLETEMENT_VACCINES),
    text=[int(round(val)) for val in y],
    textposition='outside',
    name="Totalement vaccinés"
))
    
fig.update_layout(
    barmode="group", bargroupgap=0,
    title={
                        'text': "<b>Admissions à l'hôpital</b> pour Covid",
                        'y':0.97,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'},
    titlefont = dict(
                    size=25),
    annotations = [
                        dict(
                            x=0.5,
                            y=1.12,
                            xref='paper',
                            yref='paper',
                            font=dict(size=14),
                            text="selon le statut vaccinal et l'âge, pour 10 Mio hab. de chaque groupe - {} (moyenne semaine)<br>Données DREES - @GuillaumeRozier - covidtracker.fr".format(datetime.strptime(df_drees.date.max(), '%Y-%m-%d').strftime('%d %B %Y')),#'Date : {}. Source : Santé publique France. Auteur : GRZ - covidtracker.fr.'.format(),                    showarrow = False
                            showarrow=False
                        ),
                        ]
                 )

fig.add_annotation(
    x=0.5,
    y=-0.225,
    xref='paper',
    yref='paper',
    text="<i>Une personne est considérée comme vaccinée après avoir terminé son schéma vaccinal.</i>",
    font=dict(size=9),
    showarrow=False,
    yshift=30,
    xshift=0
)

fig.update_yaxes(title="Admissions à l'hôpital pour 10 Mio")
fig.update_xaxes(title="Âge")
name_fig = "hc_statut_vaccinal_age"
fig.write_image(PATH + "images/charts/france/{}.jpeg".format(name_fig), scale=2, width=900, height=600)
fig.show()